<a href="https://colab.research.google.com/github/nekrut/BMMB554/blob/master/2021/ipynb/Pandas2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Pandas 2 | Narrow versus Wide and more

-------


> This is a reworked version of tutorial by [Justin Bois](http://justinbois.github.io/bootcamp/2020/index.html). It contains modified training datasets, adopts content to Colab environment, and includes new sections. 

In [ ]:
import numpy as np
import pandas as pd

<hr>

In the last lesson, we learned about Pandas and dipped our toe in to see its power. In this lesson, we will continue to harness the power of Pandas to pull out subsets of data we are interested in.

## Tidy data

[Hadley Wickham](https://en.wikipedia.org/wiki/Hadley_Wickham) wrote a [great article](http://dx.doi.org/10.18637/jss.v059.i10) in favor of "tidy data." Tidy data frames follow the rules:

1. Each variable is a column.
2. Each observation is a row.
3. Each type of observation has its own separate data frame.

This is less pretty to visualize as a table, but we rarely look at data in tables. Indeed, the representation of data which is convenient for visualization is different from that which is convenient for analysis. A tidy data frame is almost always **much** easier to work with than non-tidy formats.

You may raise some objections about tidy data. Here are a few, and my responses.

**Objection:** Looking at a table of tidy data is ugly. It is not intuitively organized. I would almost never display a tidy data table in a publication.

**Response:** Correct! Having tabular data in a format that is easy to read as a human studying a table is a very different thing than having it in a format that is easy to explore and work with using a computer. As [Daniel Chen](https://chendaniely.github.io) put it, "There are data formats that are better for reporting and data formats that are better for analysis." We are using the tidy data frames for analysis, not reporting (though we will see in the coming lessons that having the data in a tidy format makes making plots much easier, and plots are a key medium for reporting.)

<br />

**Objection:** Isn't it better to sometimes have data arranged in other ways? Say in a matrix?

**Response:** This is certainly true for things like images, or raster-style data in general. It makes more sense to organize an image in a 2D matrix than to have it organized as a data frame with three columns (row in image, column in image, intensity of pixel), where each row corresponds to a single pixel. For an image, indexing it by row and column is always unambiguous, `my_image[i, j]` means the pixel at row `i` and column `j`.

For other data, though, the matrix layout suffers from the fact that there may be more than one way to construct a matrix. If you know a data frame is tidy, you already know its structure. You need only to ask what the columns are, and then you immediately know how to access data using Boolean indexing. In other formats, you might have to read and write extensive comments to understand the structure of the data. Of course, you *can* read and write comments, but it opens the door for the possibility of misinterpretation or mistakes.

<br />

**Objection:** But what about time series? Clearly, that can be in matrix format. One column is time, and then subsequent columns are observations made at that time.

**Response:** Yes, that is true. But then the matrix-style described could be considered tidy, since each row is a single observation (time point) that has many facets.

<br />

**Objection:** Isn't this an inefficient use of memory? There tend to be lots of repeated entries in tidy data frames.

**Response:** Yes, there are more efficient ways of storing and accessing data. But for data sets that are not "big data," this is seldom a real issue. The extra expense in memory, as well as the extra expense in access, are small prices to pay for the simplicity and speed of the human user in accessing the data.

<br />

**Objection:** Once it's tidy, we pretty much have to use Boolean indexing to get what we want, and that can be slower than other methods of accessing data. What about performance?

**Response:** See the previous response. Speed of access really only becomes a problem with big, high-throughput data sets. In those cases, there are often many things you need to be clever about beyond organization of your data.

<br />

**Conclusion:** I really think that tidying a data set allows for fluid exploration. We will focus on tidy data sets going forward. The techniques for bringing untidy data into tidy format use many of Pandas's functions, but are largely beyond the scope of this bootcamp. You will explore that a little bit in the exercises, but for most of the bootcamp, our data sets are already tidy.

## The data set

The dataset we will be using is a list of all SARS-CoV-2 datasets in [Sequence Read Archive](https://www.ncbi.nlm.nih.gov/sra) as of January 20, 2021.  

It is obtained by going to https://www.ncbi.nlm.nih.gov/sra and performing a query with the following search terms: `txid2697049[Organism:noexp]`.

Results are downloaded using `Send to:` menu selecting `File` and then `RunInfo`. Let's get these results into this notebook:

In [ ]:
df = pd.read_csv('https://github.com/nekrut/BMMB554/raw/master/2021/data/sra_ncov_bmmb554.csv.gz')
df = df[df['size_MB']> 0].reset_index(drop=True)

# Take a look
df

This data set is in tidy format. Each row represents a single SRA dataset. The properties of each run are given in each column. We already saw the power of having the data in this format when we did Boolean indexing in the last lesson. Now, we will see how this format allows use to easily do an operation we do again and again with data sets, **split-apply-combine**.

## Split-apply-combine

Let's say we want to compute the total size of SRA runs for each BioProject. Ignoring for the second the mechanics of how we would do this with Python, let's think about it in English. What do we need to do?

1. **Split** the data set up according to the `'BioProject'` field, i.e., split it up so we have a separate data set for the two classes of subjects, those with insomnia and those without.
2. **Apply** a median function to the activity in these split data sets.
3. **Combine** the results of these averages on the split data set into a new, summary data set that contains classes for each platform and medians for each.

We see that the strategy we want is a **split-apply-combine** strategy. This idea was put forward by Hadley Wickham in [this paper](http://dx.doi.org/10.18637/jss.v040.i01). It turns out that this is a strategy we want to use *very* often. Split the data in terms of some criterion. Apply some function to the split-up data. Combine the results into a new data frame.

Note that if the data are tidy, this procedure makes a lot of sense. Choose the column you want to use to split by. All rows with like entries in the splitting column are then grouped into a new data set. You can then apply any function you want into these new data sets. You can then combine the results into a new data frame.

Pandas's split-apply-combine operations are achieved using the `groupby()` method. You can think of `groupby()` as the splitting part. You can then apply functions to the resulting `DataFrameGroupBy` object. The [Pandas documentation on split-apply-combine](http://pandas.pydata.org/pandas-docs/stable/groupby.html) is excellent and worth reading through. It is extensive though, so don't let yourself get intimidated by it.

### Aggregation: Median percent correct

Let's go ahead and do our first split-apply-combine on this tidy data set. First, we will split the data set up by insomnia condition.

In [ ]:
grouped = df.groupby(['BioProject'])

# Take a look
grouped

There is not much to see in the `DataFrameGroupBy` object that resulted. But there is a lot we can do with this object. Typing `grouped.` and hitting tab will show you the many possibilities. For most of these possibilities, the apply and combine steps happen together and a new data frame is returned. The `grouped.sum()` method is exactly what we want.

In [ ]:
df_sum = grouped.sum()

# Take a look
df_sum

The outputted data frame has the sums of numerical columns only, which we have only one: `size_MS`. Note that this data frame has `Platform` as the name of the row index. If we want to instead keep `Platform` (which, remember, is what we used to split up the data set before we computed the summary statistics) as a column, we can use the `reset_index()` method.

In [ ]:
df_sum.reset_index()

Note, though, that this was not done in-place. If you want to update your data frame, you have to explicitly do so.

In [ ]:
df_sum = df_sum.reset_index()

We can also use multiple columns in our `groupby()` operation. To do this, we simply pass in a list of columns into `df.groupby()`. We will **chain the methods**, performing a groupby, applying a median, and then resetting the index of the result, all in one line.

In [ ]:
df.groupby(['BioProject', 'Platform']).sum().reset_index()

This type of operation is called an **aggregation**. That is, we split the data set up into groups, and then computed a summary statistic for each group, in this case the median. 

## Aggregating and transforming with custom functions

Let's say we want to compute the coefficient of variation (CoV, the standard deviation divided by the mean) of data in columns of groups in the data frame. There is no built-in function to do this. We have to write our own function to compute the CoV and then use it with the `agg()` method of a `DataFrameGroupBy` object. In the function below, the values of each column are denoted by `data`.

To compute the coefficient of variation, we will use one more Numpy function beyond `np.mean()` that you have already seen, `np.std()`.

In [ ]:
def coeff_of_var(data):
    """Compute coefficient of variation from an array of data."""
    return np.std(data) / np.mean(data)

Now we can apply it as an aggregating function.

In [ ]:
grouped.agg(coeff_of_var)

You now have tremendous power in your hands. When your data are tidy, you can rapidly accelerate the ubiquitous split-apply-combine methods. Importantly, you now have a logical framework to think about how you slice and dice your data. As a final, simple example, I will show you how to go start to finish after loading the data set into a data frame, splitting by `BioProject` and `Platform`, and then getting the quartiles and extrema, in addition to the mean and standard deviation.

In [ ]:
df.groupby(['BioProject', 'Platform'])['size_MB'].describe()

In [ ]:
df.groupby(['BioProject', 'Platform'])['size_MB'].describe().reset_index()

Yes, that's right. One single, clean, easy to read line of code. In coming tutorials, we will see how to use tidy data to quickly generate plots.

## Looping over a GroupBy object

While the `GroupBy` methods we have learned so far (like `agg()`) are useful and lead to concise code, we sometimes want to loop over the groups of a `GroupBy` object. This often comes up in plotting applications, as we will see in future lessons. As an example, I will compute the median run size for diferent BioProjects stratified by Platform (which we already computed using `describe()`).

In [ ]:
for name, group in df.groupby(['BioProject', 'Platform']):
    print(name, ': ', group['size_MB'].median())

By using the `GroupBy` object as an iterator, it yields the name of the group (which I assigned as `name`) and the corresponding sub-data frame (which I assigned `group`).

## Tidying a data set

You should always organize your data sets in a tidy format. However, this is sometimes just not possible, since you data sets can come from instruments that do not output the data in tidy format (though most do, at least in my experience), and you often have collaborators that send data in untidy formats.

The most useful function for tidying data is `pd.melt()`. To demonstrate this we will use a dataset describing read coverage across SARS-CoV-2 genomes for a number of samples.

In [ ]:
df = pd.read_csv('https://github.com/nekrut/BMMB554/raw/master/2021/data/coverage.tsv',sep='\t')

df.head()

Clearly these data are not tidy. When we melt the data frame, the data within it, called **values**, become a single column. The headers, called **variables**, also become new columns. So, to melt it, we need to specify what we want to call the values and what we want to call the variable. [`pd.melt()`](https://pandas.pydata.org/docs/reference/api/pandas.melt.html#pandas.melt) does the rest!

![](https://pandas.pydata.org/docs/_images/07_melt1.svg)

> Image from [Pandas Docs](https://pandas.pydata.org/docs/getting_started/intro_tutorials/07_reshape_table_layout.html#wide-to-long-format).



In [ ]:
df = pd.melt(df, value_name='coverage', var_name=['sample'],value_vars=df.columns[3:],id_vars=['start','end'])

df.head()

...and we are good to do with a tidy DataFrame! Let's take a look at the summary. This wouild allow us to easily plot coverage:

In [ ]:
import seaborn as sns
sns.relplot(data=df, x='start',y='coverage',kind='line')

In [ ]:
import seaborn as sns
sns.relplot(data=df, x='start',y='coverage',kind='line',hue='sample')

In [ ]:
df.groupby(['sample']).describe()

In [ ]:
df.groupby(['sample'])['coverage'].describe()

To get back from melted (narrow) format to wide format we can use [`pivot()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html#pandas.DataFrame.pivot) function. 

![](https://pandas.pydata.org/docs/_images/07_pivot.svg)

> Image from [Pandas Docs](https://pandas.pydata.org/docs/getting_started/intro_tutorials/07_reshape_table_layout.html#long-to-wide-table-format).





In [ ]:
df.pivot(index=['start','end'],columns='sample',values='coverage')

## Working with multiple tables

Here we will talk about joining two tables. First, let's define two simple dataframes:



In [ ]:
!pip install --upgrade pandasql

In [ ]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [ ]:
df1 = pd.DataFrame({"key": ["A", "B", "C", "D"], "value": np.random.randn(4)})
df2 = pd.DataFrame({"key": ["B", "D", "D", "E"], "value": np.random.randn(4)})

### Inner join

Using pandas `merge`:

In [ ]:
pd.merge(df1, df2, on="key")

Using `pysqldf`:

In [ ]:
pysqldf('select * from df1 join df2 on df1.key=df2.key')

### Left join

Using pandas `merge`:

In [ ]:
pd.merge(df1, df2, on="key", how="left").fillna('.')

Using `pysqldf`:

In [ ]:
pysqldf('select * from df1 left join df2 on df1.key=df2.key').fillna('.')

In [ ]:
pysqldf('select df1.key, df1.value as value_x, df2.value as value_y from df1 left join df2 on df1.key=df2.key').fillna('.')

### Right join

Using pandas `merge`:

In [ ]:
pd.merge(df1, df2, on="key", how="right").fillna('.')

### Full join

Using pandas `merge`:

In [ ]:
pd.merge(df1, df2, on="key", how="outer").fillna('.')

## Computing environment

In [ ]:
!pip install watermark

In [ ]:
%load_ext watermark
%watermark -v -p numpy,pandas